# Artifact Sharing Notebook Version #2.
# Client of Artifact Sharing Server.
This is a client project for the Artifact Sharing project located here `https://github.com/drkiettran/artifact-sharing`.

In [1]:
import base64
import json
import http
import requests
import uuid
from Crypto.Cipher import PKCS1_OAEP

from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Cipher import PKCS1_v1_5

from base64 import b64decode, urlsafe_b64decode, b64encode, urlsafe_b64encode
from Crypto.Hash import SHA256

In [2]:
def new_id(old_id):
    new_id = old_id.split('--')[0]
    return new_id + '--' + str(uuid.uuid4())

'''
STIX id can be in these fields: id, created_by_ref, object_marking_refs, ??? Others ???
'''
def new_stix_object(stix):
    content_json = json.loads(stix)
    stix_ids = {}
    stix_ids[content_json['id']] = new_id(content_json['id'])
    
    if 'objects' in content_json:
        for obj in content_json['objects']:
            stix_ids[obj['id']] = new_id(obj['id'])
            if 'created_by_ref' in obj:
                stix_ids[obj['created_by_ref']] = new_id(obj['created_by_ref'])
            if 'object_marking_refs' in obj:
                for ref in obj['object_marking_refs']:
                    stix_ids[ref] = new_id(ref)
    else:
        obj = stix
        stix_ids[obj['id']] = new_id(obj['id'])
        if 'created_by_ref' in obj:
            stix_ids[obj['created_by_ref']] = new_id(obj['created_by_ref'])
        if 'object_marking_refs' in obj:
            for ref in obj['object_marking_refs']:
                stix_ids[ref] = new_id(ref)
    new_content = json.dumps(content_json, indent=2)
    for old_id in stix_ids:
        new_content = new_content.replace(old_id, stix_ids[old_id])
    return new_content;